# Obliczenia naukowe, lista 3
## Dominika Szydło, 250109

### Zadanie 1
Napisać funkcję rozwiązującą równanie $f(x) = 0$ metodą bisekcji.

In [2]:
function mbisekcji(f, a::Float64, b::Float64, delta::Float64, epsilon::Float64)
    #funckja rozwiązuje równanie f(x) = 0 metodą bisekcji
    #Dane:
    #      f: funkcja f(x) podana jako anonimowa funkcja
    #      a: początek przedziału
    #      b: koniec przedziały
    #  delta: dokładność przybliżenia x
    #epsilon: dokładność przybliżenia f(x)
    #
    #Wywołanie: mbisekcji(x->x^2-5, -5.0, 0.0, 0.0001, 0.00001)
    #
    #Wynik: czwórka (r, v, it, err) gdzie
    #     r: przybliżenie rozwiązania równania f(x) = 0
    #     v: f(r)
    #    it: liczba iteracji wykonanych przez algorytm
    #   err: sygnalizacja błędu:
    #        0: brak błędu
    #        1: f nie zmienia znaku na przedziale [a,b]
    c::Float64 = 0.0
    w::Float64 = 0.0
    u = f(a)
    v = f(b)
    e = b-a
    it = 0
    if signbit(u) == signbit(v)
        return (c, w, it, 1)
    end
    while e > delta
        it += 1
        e /= 2.0
        c = a+e
        w = f(c)
        if abs(e) < delta || abs(w) < epsilon
            return (c, w, it, 0)
        end
        if signbit(w) != signbit(u)
            b = c
            v = w
        else
            a = c
            u = w
        end
    end
end

mbisekcji (generic function with 1 method)

**Opis algorytmu**

Metoda bisekcji opiera sie na tw. Darboux, które mówi, że jeżeli dla funkcji ciągłej $f: [a,b] \longrightarrow \mathbb {R}$ zachodzi $f(a)*f(b) < 0$ ($f$ zmienia znak na przedziale $[a, b]$), to w przedziale $(a, b)$ istnieje takie $c$, że $f(c) = 0$. Warunek ten jest sprawdzany w pierwszym kroku algoytmu. Jeżeli dane wejściowe go nie spełniają, to funkcja zwraca kod błędu 1, a jeżeli spełniają, to funkcja przechodzi do pętli `while`. W kolejnych iteracjach wyliczany jest środek przedziału $[a, b]$, jednak nie z potencjalnie problematycznego wzoru $\frac{a+b}{2}$, a poprzez wyliczenie początkowej długości przedziału i sukcesywne dzielenie jej przez $2$. Następnie wyliczana jest wartość funkcji $f$ dla środka przedziału - jeżeli jest ona mniejsza niż zadany $\epsilon$ lub jeżeli długość aktualnie rozpatrywanego przedziału jest mniejsza niż $2\delta$, to funkcja zwraca bieżący środek przedziału jako rozwiązanie równania. Jeżeli nie, to funkcja przechodzi do lewej lub prawej połowy przedziału, w zależności od tego która z nich spełnia tw. Darboux i przechodzi do kolejnej iteracji. Sprawdzenie spełniania tw. zachodzi przez porównanie znaków wartości funkcji na lewym końcu przedziału i na środku, a nie przez badanie ich iloczynu, ponieważ mogłoby to prowadzić do błędów związanych z liczeniem w arytmetyce zmiennoprzecinkowej.

### Zadanie 2

Napisać funkcję rozwiązującą równanie $f(x) = 0$ metodą Newtona.

In [3]:
function mstycznych(f,pf,x0::Float64, delta::Float64, epsilon::Float64, maxit::Int)
    #funkcja rozwiązuje równanie f(x) = 0 metodą Newtona
    #Dane:
    #      f: funckja f(x) podana jako anonimowa funkcja
    #     pf: pochodzna f(x) podana jako anonimowa funkcja
    #     x0: przybliżenie początkowe
    #  delta: dokładność przybliżenia x
    #epsilon: dokładność przybliżenia f(x)
    #  maxit: maksymalna dopuszczalna liczba iteracji
    #
    #Wywołanie: mstycznych(x->x^2-5, x->2x, -4.0, 0.00001, 0.00001, 5)
    #
    #Wynik: czwórka (r, v, it, err) gdzie
    #      r: przybliżenie rozwiązania równania f(x) = 0
    #      v: f(r)
    #     it: liczba iteracji wykonanych przez algorytm
    #    err: sygnalizacja błędu
    #         0: nie wystąpił błąd
    #         1: nie osiągnięto wymaganej dokładności w maxit iteracji
    #         2: pochodna bliska 0
    it = 0
    x1::Float64 = 0.0
    v = f(x0)
    if abs(pf(x0)) < epsilon
        return (x1, f(x1), it, 2)
    end
    if abs(v) < epsilon
        return (x0, v, it, 0)
    end
    for k in 0:maxit
        x1 = x0-v/pf(x0)
        v = f(x1)
        if abs(x1-x0) < delta || abs(v) < epsilon
            return (x1, v, k+1, 0)
        end
        x0 = x1
    end
    return (x1, v, maxit, 1)
end        

mstycznych (generic function with 1 method)

**Opis algorytmu**

Metoda stycznych korzysta z linearyzacji funkcji za pomocą dwóch pierwszych wyrażeń w szeregu Taylora. Wtedy $f(x) \approx f(x_n) + f'(x_n)(x-x_n)$. Kolejne wartości $x_n$ obliczane są ze wzoru $x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}$, stąd wymóg podania $x_0$ jako argument funkcji oraz założenie, że $f'(x_0) \neq 0$ (czyli $|f'(x_0)| > \epsilon$, z uwagi na ograniczenia arytmetyki). Jeżeli wymóg ten nie jest spełniony, to funkcja zwraca kod błędu $2$ i kończy pracę, a w przeciwnym przypadku przechodzi do sprawdzenia czy przypadkiem $f(x_0)$ nie jest na tyle blisko $0$, by zwrócić $x_0$ jako wynik. Jeżeli nie, to funkcja przechodzi do pętli `for`, która wykonuje się conajwyżej `maxit` razy. W niej wyliczane są kolejne wartości $x$, aż do momentu, gdy odległość między $x_n$ a $x_{n+1}$ będzie mniejsza niż $\delta$ lub $f(x_{n+1})$ będzie wystarczająco blisko $0$, lub gdy zostanie wykorzystany limit iteracji. Jeżeli tak się stanie, to funkcja zwróci ostatnie $x_{n+1}$ i kod błędu $1$.

### Zadanie 3

Napisać funkcję rozwiązującą równanie $f(x) = 0$ metodą siecznych.

In [4]:
function msiecznych(f, x0::Float64, x1::Float64, delta::Float64, epsilon::Float64, maxit::Int)
    #funkcja rozwiązuje równanie f(x) = 0 metodą siecznych
    #Dane:
    #      f: funckja f(x) podana jako anonimowa funkcja
    # x0, x1: przybliżenia początkowe
    #  delta: dokładność przybliżenia x
    #epsilon: dokładność przybliżenia f(x)
    #  maxit: maksymalna dopuszczalna liczba iteracji
    #
    #Wywołanie: msiecznych(x->x^2-5, -4.5, -4.0, 0.00001, 0.00001, 5)
    #
    #Wynik: czwórka (r, v, it, err) gdzie
    #      r: przybliżenie rozwiązania równania f(x) = 0
    #      v: f(r)
    #     it: liczba iteracji wykonanych przez algorytm
    #    err: sygnalizacja błędu
    #         0: nie wystąpił błąd
    #         1: nie osiągnięto wymaganej dokładności w maxit iteracji
    f0 = f(x0)
    f1 = f(x1)
    for k in 0:maxit
        if abs(f0) > abs(f1)
            x0, x1 = x1, x0
            f0, f1 = f1, f0
        end
        s = (x1-x0)/(f1-f0)
        x1 = x0
        f1 = f0
        x0 -= f0*s
        f0 = f(x0)
        if abs(x1-x0) < delta || abs(f0) < epsilon
            return (x0, f0, k+1, 0)
        end
    end
    return (x0, f0, maxit, 1)
end

msiecznych (generic function with 1 method)

**Opis algorytmu**

Metoda siecznych również korzysta z pochodnej funkcji $f$, ale w postaci ilorazu różnicowego $f'(x) \approx \frac{f(x_n)-f(x_{n-1})}{x_n-x_{n-1}}$, dzięki czemu nie trzeba podawać jej jako argument funkcji. Kolejne wartości $x_n$ obliczane są ze wzoru $x_{n+1} = x_n - \frac{x_n-x_{n-1}}{f(x_n)-f(x_{n-1})}f(x_n)$, stąd wymóg podania $x_0$ oraz $x_1$ w wywołaniu funkcji. W pętli `for` algorytmu na początku sprawdzane jest dla którego z $x$-ów wartość bezwzględna z $f(x)$ przyjmuje mniejszą wartość oraz ustawienie ich niemalejąco według tego parametru. Jest to konieczne, by parametr ten malał dla kolejnych $x$-ów, a tym samym by zbliżały się do rozwiązania równania. Następnie wyliczany jest $x_{n+1}$ i wartość funkcji $f$ jaką dla niego przyjmuje. Później sprawdzane jest jak blisko siebie są $x_{n+1}$ i $x_{n}$ i jak daleko od $0$ odbiega $f(x_{n+1})$ - jeżeli któraś z tych wartości jest mniejsza odpowiednio od $\delta$ lub $\epsilon$, to algorytm zwraca $x_{n+1}$ jako rozwiązanie równania. Jeżeli nie, to pętla jest kontynuowana aż do `maxit` iteracji. Jeśli wciąż nie udało się znaleźć rozwiązania to funkcja zwraca ostatni $x_{n+1}$ jako odpowiedź i kod błędu $1$.

**Okienko testowe:**

In [14]:
f(x) = x^2-1
pf(x) = 2*x
a = -3.0
b = -0.5
maxit = 100
x0 = 40.0
x1 = 2000.0
delta = 0.00001
epsilon = 0.00001
println("Wyniki uzyskane za pomocą metody:")
println("- bisekcji:  $(mbisekcji(f, a, b, delta, epsilon))")
println("- Newtona:   $(mstycznych(f, pf, x0, delta, epsilon, maxit))")
println("- siecznych: $(msiecznych(f, x0, x1, delta, epsilon, maxit))")

Wyniki uzyskane za pomocą metody:
- bisekcji:  (-1.0000019073486328, 3.814700903603807e-6, 18, 0)
- Newtona:   (1.0000000000151628, 3.0325519873031226e-11, 9, 0)
- siecznych: (1.000000010272644, 2.0545288137441275e-8, 13, 0)


### Zadanie 4

Wyznaczyć pierwiastek równania $sin(x) - (\frac{1}{2}x)^2 = 0$ za pomocą wcześniej napisanych funkcji.

In [6]:
println("Wyniki uzyskane za pomocą metody:")
println("- bisekcji:  $(mbisekcji(x->sin(x)-(0.5*x)^2, 1.5, 2.0, 0.5*0.00001, 0.5*0.00001))")
println("- Newtona:   $(mstycznych(x->sin(x)-(0.5*x)^2, x->cos(x)-0.5*x, 1.5, 0.5*0.00001, 0.5*0.00001, 5))")
println("- siecznych: $(msiecznych(x->sin(x)-(0.5*x)^2, 1.0, 2.0, 0.5*0.00001, 0.5*0.00001, 5))")

Wyniki uzyskane za pomocą metody:
- bisekcji:  (1.9337539672851562, -2.7027680138402843e-7, 16, 0)
- Newtona:   (1.933753779789742, -2.2423316314856834e-8, 4, 0)
- siecznych: (1.933753644474301, 1.564525129449379e-7, 4, 0)


Metody Newtona i siecznych wykonały $4$-krotnie mniej iteracji niż metoda bisekcji. Dzieje się tak, ponieważ wykładniki zbieżności metody Newtona i siecznych wynoszą odpowiednio $2$ i $\frac{1+\sqrt{5}}{2} \approx 1.618...$, tymczasem dla metody bisekcji parametr ten wynosi $1$, a więc zbiega ona do dobrego rozwiązania o wiele wolniej niż pozostałe metody. Metoda Newtona również zwróciła $x$ dla którego wartość $f(x)$ jest najbliżej $0$. 

### Zadanie 5

Metodą bisekcji znaleźć wartości zmiennej $x$, dla której przecinają się wykresy funkcji $y = 3x$ i $y = e^x$.

Aby znaleźć odpowiedni przedział $[a, b]$ wygenerowałam w programie Desmos wykres funkcji $f(x) = 3x - e^x$ (jest to funkcja, którą będę rozważać).

![wykres](./wykres.png)

In [7]:
println("x_1: $(mbisekcji(x->3*x-ℯ^x, 0.5, 0.8, 0.0001, 0.0001))")
println("x_2: $(mbisekcji(x->3*x-ℯ^x, 1.5, 1.6, 0.0001, 0.0001))")

x_1: (0.6190917968749999, 3.486661493345977e-5, 11, 0)
x_2: (1.512109375, 3.868007140983565e-5, 8, 0)


Porównując z wynikami z programu Wolfram Alpha można stwierdzić, że metoda zwróciła wartości zgodne z zadanymi dokładnościami.

![x1](./x1.png)
![x2](./x2.png)

### Zadanie 6

Znaleźć miejsce zerowe funkcji $f_1(x) =e^{1−x}−1$ oraz $f_2(x) = xe^{−x}$ za pomocą metod bisekcji, Newtona i siecznych.

Rozwiązaniem równania $f_1(x) = 0$ jest $x = 1$, natomiast $f_2(x) = 0$ jest $x = 0$.

In [8]:
println("Wyniki dla f1 metodą bisekcji dla przedziałów:")
println("[-2.0, 1.3]    - $(mbisekcji(x->ℯ^(1-x)-1, -2.0, 1.3, 0.00001, 0.00001))")
println("[-2.0, 2.0]    - $(mbisekcji(x->ℯ^(1-x)-1, -2.0, 2.0, 0.00001, 0.00001))")
println("[-15.0, 2.0]   - $(mbisekcji(x->ℯ^(1-x)-1, -15.0, 2.0, 0.00001, 0.00001))")
println("[-34.7, 52.25] - $(mbisekcji(x->ℯ^(1-x)-1, -34.7, 52.25, 0.00001, 0.00001))")

Wyniki dla f1 metodą bisekcji dla przedziałów:
[-2.0, 1.3]    - (0.9999908447265624, 9.15531534717573e-6, 15, 0)
[-2.0, 2.0]    - (1.0, 0.0, 2, 0)
[-15.0, 2.0]   - (0.9999990463256836, 9.536747711536009e-7, 20, 0)
[-34.7, 52.25] - (1.000006949901579, -6.949877428441553e-6, 22, 0)


Metoda zwróciła poprawne wyniki dla każdego z przedziałów, nie ważne jak długiego - jest dlatego, bo jest zbieżna globalnie. W przypadku przedziału $[-2.0, 2.0]$ zwróciła dokładny wynik, ponieważ $x = 1.0$ znajduje się na środku przedziału $[0.0, 2.0]$, do którego algorytm przechodzi po pierwszej iteracji.

In [9]:
println("Wyniki dla f1 metodą Newtona dla maxit = 100:")
println("x0 = 0.5  - $(mstycznych(x->ℯ^(1-x)-1, x->-ℯ^(1-x), 0.5, 0.00001, 0.00001, 100))")
println("x0 = -5.0 - $(mstycznych(x->ℯ^(1-x)-1, x->-ℯ^(1-x), -2.0, 0.00001, 0.00001, 100))")
println("x0 = 4.0  - $(mstycznych(x->ℯ^(1-x)-1, x->-ℯ^(1-x), 4.0, 0.00001, 0.00001, 100))")
println("x0 = 10.0 - $(mstycznych(x->ℯ^(1-x)-1, x->-ℯ^(1-x), 10.0, 0.00001, 0.00001, 100))")

Wyniki dla f1 metodą Newtona dla maxit = 100:
x0 = 0.5  - (0.9999999998878352, 1.1216494399945987e-10, 4, 0)
x0 = -5.0 - (0.9999999999251376, 7.48623385504743e-11, 7, 0)
x0 = 4.0  - (0.9999999995278234, 4.721765201054495e-10, 21, 0)
x0 = 10.0 - (NaN, NaN, 100, 1)


Metoda w trzech pierwszych wywołaniach zwraca poprawne wyniki. O ile dla dwóch pierwszych znajduje je stosunkowo szybko, to dla $x_0 = 4.0$ liczba iteracji znacznie się zwiększa, a w przypadku $x_0 = 10.0$ otrzymujemy kod błędu $1$, oznaczający brak znalezienia rozwiązania w `maxit` iteracjach, a jako wynik `Nan`. Dzieje się tak, ponieważ w metodzie Newtona wykorzystujemy wzór $x_{n+1}=x_n-\frac{f(x_n)}{f'(x_n)}$, a dla $f(x) = e^{1-x}-1$ wyrażenie $\frac{f(x_n)}{f'(x_n)}$ przyjmuje postać $\frac{e^{1-x}-1}{-e^{1-x}}$. Mianownik dla $x > 5$ jest bardzo bliski $0$, przez występuje błąd reprezentacji w arytmetyce, w wyniku którego dostajemy $0$ w mianowniku, a jako wynik `NaN`. Metoda Newtona jest również zbieżna lokalnie, a nie globalnie.

In [10]:
println("Wyniki dla f1 metodą siecznych dla maxit = 100:")
println("x0 = -1.0, x1 = 0.0  - $(msiecznych(x->ℯ^(1-x)-1, -1.0, 0.0, 0.00001, 0.00001, 100))")
println("x0 = -2.0, x1 = 1.5  - $(msiecznych(x->ℯ^(1-x)-1, -2.0, 1.5, 0.00001, 0.00001, 100))")
println("x0 = -2.0, x1 = -3.0 - $(msiecznych(x->ℯ^(1-x)-1, -2.0, -3.0, 0.00001, 0.00001, 100))")
println("x0 = 10.0, x1 = 11.0 - $(msiecznych(x->ℯ^(1-x)-1, 10.0, 11.0, 0.00001, 0.00001, 100))")

Wyniki dla f1 metodą siecznych dla maxit = 100:
x0 = -1.0, x1 = 0.0  - (0.9999990043764041, 9.956240916153547e-7, 6, 0)
x0 = -2.0, x1 = 1.5  - (1.0000000218730498, -2.1873049593779115e-8, 6, 0)
x0 = -2.0, x1 = -3.0 - (0.9999970015876295, 2.9984168656849164e-6, 9, 0)
x0 = 10.0, x1 = 11.0 - (10.0, -0.9998765901959134, 2, 0)


Metoda siecznych zachowuje się podobnie do metody Newtona - im przybliżenia początkowe są bliższe do rozwiązania faktycznego i bliższe do siebie nawzajem, tym lepsze jest przybliżenie oraz jego znalezienie wymaga mniej iteracji. Jest tak, ponieważ metoda ta również jest zbieżna lokalnie. W przypadku wybrania odległych przybliżeń od rzeczywistego rozwiązania możemy otrzymać możemy otrzymać zarówno dobre jak i złe wyniki, wszystko jest zależne od monotoniczności badanej funkcji.

In [11]:
println("Wyniki dla f2 metodą bisekcji dla przedziałów:")
println("[-2.0, 1.3]    - $(mbisekcji(x->x*ℯ^(-x), -2.0, 1.3, 0.00001, 0.00001))")
println("[-2.0, 2.0]    - $(mbisekcji(x->x*ℯ^(-x), -2.0, 2.0, 0.00001, 0.00001))")
println("[-15.0, 2.0]   - $(mbisekcji(x->x*ℯ^(-x), -15.0, 2.0, 0.00001, 0.00001))")
println("[-34.7, 52.25] - $(mbisekcji(x->x*ℯ^(-x), -34.7, 52.25, 0.00001, 0.00001))")

Wyniki dla f2 metodą bisekcji dla przedziałów:
[-2.0, 1.3]    - (-1.907348632920158e-6, -1.9073522709024344e-6, 18, 0)
[-2.0, 2.0]    - (0.0, 0.0, 1, 0)
[-15.0, 2.0]   - (-1.9073486328125e-6, -1.9073522707947765e-6, 19, 0)
[-34.7, 52.25] - (9.310245512208098e-6, 9.310158831940109e-6, 22, 0)


Podobnie jak dla $f_1$, metoda bisekcji zwróciła wyniki zgodne z zadaną dokładnością.

In [12]:
println("Wyniki dla f2 metodą Newtona dla maxit = 100:")
println("x0 = -2.5 - $(mstycznych(x->x*ℯ^(-x), x->-ℯ^(-x)*(x-1), -2.5, 0.00001, 0.00001, 100))")
println("x0 = -5.0 - $(mstycznych(x->x*ℯ^(-x), x->-ℯ^(-x)*(x-1), -5.0, 0.00001, 0.00001, 100))")
println("x0 = 1.0  - $(mstycznych(x->x*ℯ^(-x), x->-ℯ^(-x)*(x-1), 1.0, 0.00001, 0.00001, 100))")
println("x0 = 5.0  - $(mstycznych(x->x*ℯ^(-x), x->-ℯ^(-x)*(x-1), 5.0, 0.00001, 0.00001, 100))")

Wyniki dla f2 metodą Newtona dla maxit = 100:
x0 = -2.5 - (-3.3084197593330218e-6, -3.3084307049924325e-6, 7, 0)
x0 = -5.0 - (-9.064102913053547e-6, -9.064185071387511e-6, 10, 0)
x0 = 1.0  - (0.0, 0.0, 0, 2)
x0 = 5.0  - (15.194283983439147, 3.827247505782993e-6, 9, 0)


Dla dwóch pierwszych wywołań metoda Newtona zwraca poprawne wyniki. Nie można wybrać $x_0 = 1$, ponieważ otrzymujemy kod błędu $2$, bo $f'_2(1) = 0$. Dla $x_0 = 5.0$ metoda zwróciła wynik poprawny względem dokładności przybliżenia wartości $f(x)$, jednak sam $x$ jest bardzo odległy od faktycznego rozwiązania równania. Dla tej metody dobór parametrów początkowych ma bardzo istotny wpływ na ostateczny wynik.

In [13]:
println("Wyniki dla f2 metodą siecznych dla maxit = 100:")
println("x0 = -2.0, x1 = -1.0 - $(msiecznych(x->x*ℯ^(-x), -2.0, -1.0, 0.00001, 0.00001, 100))")
println("x0 = -3.0, x1 = -2.0 - $(msiecznych(x->x*ℯ^(-x), -3.0, -2.0, 0.00001, 0.00001, 100))")
println("x0 = 1.0, x1 = 2.0   - $(msiecznych(x->x*ℯ^(-x), 1.0, 2.0, 0.00001, 0.00001, 100))")
println("x0 = 4.0, x1 = 5.0   - $(msiecznych(x->x*ℯ^(-x), 4.0, 5.0, 0.00001, 0.00001, 100))")

Wyniki dla f2 metodą siecznych dla maxit = 100:
x0 = -2.0, x1 = -1.0 - (-6.982568902521766e-6, -6.982617658960467e-6, 7, 0)
x0 = -3.0, x1 = -2.0 - (-2.466940303935572e-8, -2.4669403647935173e-8, 10, 0)
x0 = 1.0, x1 = 2.0   - (14.787653201811631, 5.593750238217686e-6, 14, 0)
x0 = 4.0, x1 = 5.0   - (14.445548632765878, 7.693287836872918e-6, 12, 0)


Również dla $f_2$ metoda siecznych zachowuje się podobnie do metody Newtona.